*This notebook can be found on* [github](https://github.com/qojulia/QuantumOptics.jl-examples/tree/master/notebooks/homodyne-master.ipynb)

# Quantum Zeno Effect

In this example, we will use a stochastic master equation approach in order to describe the dynamics of an atom inside a cavity, where the cavity is subject to homodyne detection. The example was inspired by the work of Nielsen and Mølmer [1] and we reproduce some of their results.

The model is as follows: a number of identical three-level atoms in a $\Lambda$ configuration are placed inside a cavity. Each atom has two ground states $|f\rangle$ and $|g\rangle$. The cavity couples to the transition between the excited state and $|f\rangle$, but is so far detuned that the excited state may be adiabatically eliminated. Hence, the only remaining dynamics are a shift of the cavity field caused by all atoms that are in the state $|f\rangle$. Since all atoms are identical, we can describe them by a Dicke basis. This system can be modelled by the Hamiltonian

$H = g a^\dagger a ~n,$

where $n$ is the number operator counting the number of atoms in state $|f\rangle$. In addition to the interaction between the cavity and the atoms, we add a transversal pump that coherently drives the atoms from $|f\rangle$ to $|g\rangle$,

$H_\text{p} = g_s\sum_i\left(\sigma_i^+ + \sigma_i^-\right),$

where $g_s$ is the amplitude of the coherent drive, $\sigma_i^+ = |f\rangle_i \langle g|_i$ and $\sigma_i^-=\left(\sigma_i^+\right)^\dagger$. Note, that the atomic raising and lowering operators can be rewritten as Dicke ladder operators. Furthermore, the cavity decay is accounted for by the damping rate $\kappa=\kappa_1+\kappa_2$. Probe light is fed into the cavity with $\kappa_1$ and the detector is set up at the output channel with $\kappa_2$. For further details, please see Ref. [1].

We proceed as usual by loading the needed libraries, defining the parameters and the Hilbert space and the operators of the system.

In [ ]:
using QuantumOptics
using PyPlot

In [ ]:
# Parameters
Nc = 3
Na = 1
κ = 1.0
κ₁ = 0.5κ
κ₂ = 0.5κ
g = 0.2κ
β = 0.2*sqrt(κ)
gₛ = 1e-2κ
T = [0:0.1:3/gₛ;]

# Basis and operators
bc = FockBasis(Nc)
ba = SpinBasis(Na//2)

a = destroy(bc) ⊗ one(ba)
ad = create(bc) ⊗ one(ba)
σ⁺ = one(bc) ⊗ sigmap(ba)
σ = one(bc) ⊗ sigmam(ba)
n = one(bc) ⊗ (sigmaz(ba) + Na*one(ba))/2.0;

The linear stochastic master equation for detection at multiple outputs can, in general, be written as [2]

$\dot{\rho} = -i[H,\rho] + \sum_n\left(\mathcal{D}[c_n]\rho + \mathcal{H}[c_n]\rho \xi(t)\right),$

where $c_n$ is an arbitrary damping operator and

$\mathcal{D}[c]\rho = c\rho c^\dagger - \frac{1}{2}\left(c^\dagger c\rho - \rho c^\dagger c\right)$

is the Lindblad term,

$\mathcal{H}[c]\rho = c\rho + \rho c^\dagger$

is the measurement operator and $\xi(t)=dW(t)/dt$ is white noise.

This is the form of the stochastic master equation that is implemented with the `stochastic.master` function.

However, the equation in this form does not generally conserve the trace of the density operator. This is only the case if the damping operators $c$ fulfill certain properties. We can, though, renormalize the density matrix by adding the term  $- \langle c + c^\dagger\rangle\rho$ to $\mathcal{H}[c]$. For a nice and rigorous derivation of stochastic master equations in the above form, please see Ref. [2].

In order to implement this we resort to using the implemented dynamic version of the stochastic master equation, `stochastic.master_dynamic`. We proceed by defining the Hamiltonian and jump operators as usual. To treat the case presented in Ref. [1], we set $c\to \sqrt{\kappa}a$ in the Lindblad term $\mathcal{D}$ and $c\to -i\sqrt{\kappa_2}a$ for the measurement operator.

In [ ]:
# Hamiltonian
H0 = g*ad*a*n + gₛ*(σ⁺ + σ)

# Coherent feeding with probe light
Hf = -1.0im*sqrt(κ₁)*(β*ad - conj(β)*a)

H = H0 + Hf

# Damping operators of master equation
J = [a]
Jdagger = dagger.(J)
rates = [κ];

Now, we define the stochastic jump operators, which are to be applied as shown above for $\mathcal{H}[c]$.

In [ ]:
# Stochastic damping operators
Js = [-1.0im*a]
rates_s = [sqrt(κ₂)]
Jsdagger = dagger.(Js);

In [ ]:
# Initial state
ρ0 = dm(fockstate(bc, 0) ⊗ spindown(ba));

In [ ]:
# Deterministic function for standard dynamic master equation
function fdeterm(t, rho)
    H, J, Jdagger, rates
end;

Now, we will additionally define the stochastic function where we add the trace-normalizing expectation value to $\mathcal{H}$.

In [ ]:
# Compute outside of function for efficiency
id = one(a)
X = Js + Jsdagger

# Define function with expectation value to conserve trace
function fstoch(t, rho)
    Js, Jsdagger - [expect(x, rho)*id for x=X]
end;

Finally, we pass the above definitions to the stochastic, dynamic master equation in order to obtain our results.

In [ ]:
# Solve stochastic master equation
dt = 1e-3
tout, ρt = stochastic.master_dynamic(T, ρ0, fdeterm, fstoch; rates_s=rates_s, dt=dt);

In addition, we calculate the same time evolution once more, but with stronger measurement back action. Note, that as opposed to [1], we increase the back action, $\kappa_2$, rather than decreasing the pump strength $g_s$. This is simply to keep computational time to a minimum.

In [ ]:
κ₂_zeno = 40κ
rates_zeno = [sqrt(κ₂_zeno)]
tout, ρt_zeno = stochastic.master_dynamic(T, ρ0, fdeterm, fstoch; rates_s=rates_zeno, dt=dt);

As reference, we also calculate the time evolution where the coupling to the cavity is zero, i.e. the atoms will simply oscillate between $|f\rangle$ and $|g\rangle$.

In [ ]:
# Compute time evolution with no coupling to the cavity
tout2, ρt_det = timeevolution.master(T, ρ0, H-g*ad*a*n, J; rates=rates);

In [ ]:
# Calculate expectation value of atoms in |f>
proj = one(bc) ⊗ dm(spindown(ba))
p0 = expect(proj, ρt)
p0_zeno = expect(proj, ρt_zeno)
p0_det = expect(proj, ρt_det)

figure(figsize=(8, 3))
plot(tout.*gₛ, p0, label="Low Zeno")
plot(tout.*gₛ, p0_zeno, label="High Zeno")
plot(tout2.*gₛ, p0_det, ls="dashed", label="No Zeno")
axis([0, T[end]*gₛ, 0, 1.1])
xlabel(L"$g_s t$")
ylabel(L"$\langle 0|\rho|0\rangle$")
legend();

We can clearly see, that while the unperturbed atom undergoes a full Rabi cycle from $|f\rangle$ to $|g\rangle$ and back, that as soon as the system is subject to measurement noise, the probability of finding the atom in the state $|f\rangle$ is decreased. If the back action is very large, then the probability almost vanishes. This is called the Quantum Zeno effect.

**References**

[1] Nielsen, A. E. B. and Mølmer, K. *Stochastic master equation for a probed system in a cavity*, Phys. Rev. A 77, 052111, (2008). URL: https://arxiv.org/abs/0802.1225

[2] Jacobs, K. and Steck, D. A. *A straighforward introduction to continuous quantum measurements*, Contemporary Physics, 47:5, 279-303, (2006). URL: https://arxiv.org/abs/quant-ph/0611067